<h1 align="center"><b>KOTAK SALESIAN SCHOOL</b></h1>


<h2 align="center"><b>STUDENTS DATABASE MANAGEMENT</b></h2>

## **Import Required Libraries**

In [1]:
import os
import datetime
import subprocess
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sqlalchemy import create_engine, text
from oauth2client.service_account import ServiceAccountCredentials
from sqlalchemy.exc import SQLAlchemyError
import urllib.parse

## **Backup Files Before running New**

In [2]:
# * PostgreSQL Credentials
DB_USER = "postgres"
DB_PASSWORD = "Hari@123"
DB_NAME = "kotakschooldb"
DB_HOST = "localhost"
DB_PORT = "5432"
BACKUP_DIR = "D:/postgres_backups"  # * Backup directory

# * Full path to pg_dump (if needed)
PG_DUMP_PATH = r"C:\Program Files\PostgreSQL\17\bin\pg_dump.exe"

# * Ensure the backup directory exists
os.makedirs(BACKUP_DIR, exist_ok=True)

# * Generate a timestamp for the backup file
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
backup_file = os.path.join(BACKUP_DIR, f"backup_{DB_NAME}_{timestamp}.sql")

# * Run pg_dump
try:
    result = subprocess.run(
        [
            PG_DUMP_PATH,  # Use full path if not in PATH
            "-U", DB_USER,
            "-h", DB_HOST,
            "-p", DB_PORT,
            "-F", "c",
            "-b",
            "-v",
            "-f", backup_file,
            DB_NAME
        ],
        env={**os.environ, "PGPASSWORD": DB_PASSWORD},
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )

    # * Check for errors
    if result.returncode == 0:
        print(f"✅ Backup successful: {backup_file}")
    else:
        print(f"❌ Backup failed!\nError: {result.stderr}")

except FileNotFoundError:
    print(f"⚠️ pg_dump not found at {PG_DUMP_PATH}. Check PostgreSQL installation or system PATH.")

except Exception as e:
    print(f"⚠️ An unexpected error occurred: {e}")


✅ Backup successful: D:/postgres_backups\backup_kotakschooldb_2025-07-27_20-09-31.sql


## **Import Libraries & Define Credentials**

In [3]:
# === CONFIG ===
GOOGLE_JSON_PATHS = {
    "2024-25": r"D:\GITHUB\kotak-school-dbms\google_api_keys\woven-solution-446513-f2-2024-25.json",
    "2025-26": r"D:\GITHUB\kotak-school-dbms\google_api_keys\woven-solution-446513-f2-2025-26.json",
}

GOOGLE_SHEET_TITLES = {
    "2024-25": "STUDENTS DETAILS 2024-25",
    "2025-26": "STUDENTS DETAILS 2025-26",
}

UNIQUE_KEY = "Adm No."

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "kotakschooldb",
}

TABLE_NAME = "students"

## **Extract Data from Google Sheet**

In [4]:
# === FETCH GOOGLE SHEET ===
def fetch_data(sheet_title, worksheet_name="Overall", json_path=None):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
    client = gspread.authorize(creds)
    spreadsheet = client.open(sheet_title)
    sheet = spreadsheet.worksheet(worksheet_name)
    data = sheet.get_all_records(head=3)
    return pd.DataFrame(data)

# === CLEAN COLUMN NAMES ===
def clean_column_names(df):
    df.columns = df.columns.str.strip()
    return df

In [5]:
# === MERGE AND TAG STATUS ===
def merge_and_tag():
    df_2024 = clean_column_names(fetch_data(GOOGLE_SHEET_TITLES["2024-25"], "Overall", GOOGLE_JSON_PATHS["2024-25"]))
    df_2025 = clean_column_names(fetch_data(GOOGLE_SHEET_TITLES["2025-26"], "Overall", GOOGLE_JSON_PATHS["2025-26"]))

    df_2024["academic_year"] = "2024-25"
    df_2025["academic_year"] = "2025-26"

    codes_2024 = set(df_2024[UNIQUE_KEY])
    codes_2025 = set(df_2025[UNIQUE_KEY])

    continuing = codes_2024.intersection(codes_2025)
    left = codes_2024 - codes_2025
    new = codes_2025 - codes_2024

    df_2024["status"] = df_2024[UNIQUE_KEY].apply(lambda x: "left" if x in left else "continuing")
    df_2025["status"] = df_2025[UNIQUE_KEY].apply(lambda x: "new" if x in new else "continuing")

    return pd.concat([df_2024, df_2025], ignore_index=True)

In [6]:
def clean_data(df):
    df = df.copy()

    # ✨ Rename columns to match your database structure
    df.columns = [
        "sno", "adm_no", "name", "class", "gender", "mother_name", "father_name",
        "pen_number", "dob", "phone_no", "religion", "caste", "sub_caste",
        "second_lang", "remarks", "class_nos", "joined_year", "grades",
        "academic_year", "status"
    ]

    # Lowercase and strip spaces for consistency
    df.columns = df.columns.str.strip().str.lower()

    # 🗓️ Convert DOB to MySQL/PostgreSQL date format
    df["dob"] = pd.to_datetime(df["dob"], format="%d-%m-%Y", errors='coerce').dt.strftime("%Y-%m-%d")

    # 🔢 Convert joined_year to integer
    df["joined_year"] = pd.to_numeric(df["joined_year"], errors="coerce").astype("Int64")

    # 🧹 Optional: remove known junk column
    if "apaar_status" in df.columns:
        df.drop(columns=["apaar_status"], inplace=True)

    df = df.sort_values(by=["academic_year", "class_nos", "gender","name"  ], ascending=[True,  True, True, True])

    df["gender"] = df["gender"].str.upper()

    df["sno"] = range(1, len(df) + 1)

    # 🟢 Ensure no NaN values in non-nullable fields
    df["adm_no"] = df["adm_no"].astype(str).str.strip()
    df["name"] = df["name"].astype(str).str.strip()
    df["academic_year"] = df["academic_year"].astype(str).str.strip()

    # 🧾 Save to CSV for audit/debug
    output_path = r"D:\GITHUB\kotak-school-dbms\output_data\students_data.csv"
    df.to_csv(output_path, index=False)
    print(f"✅ Cleaned data saved to {output_path}")

    return df

In [7]:
def update_database(df):
    import numpy as np
    password = urllib.parse.quote_plus(POSTGRES_CREDENTIALS["password"])
    engine = create_engine(
        f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
        f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}",
        echo=False
    )

    try:
        with engine.begin() as conn:
            # Step 1: Truncate the table
            conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME} RESTART IDENTITY CASCADE;"))
            print(f"🗑️ Old records deleted from '{TABLE_NAME}'.")

        # Step 2: Prepare DataFrame for PostgreSQL
        df = df.replace({pd.NA: None, np.nan: None})

        print("⏳ Inserting data into DB...")

        # Step 3: Insert with chunking to prevent timeout
        df.to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False, method='multi', chunksize=500)

        # Step 4: Confirm insert count
        with engine.connect() as conn:
            result = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME};"))
            count = result.scalar()
            print(f"✅ Insert complete. 📊 Table now contains {count} records.\n")

    except Exception as e:
        print(f"❌ Error updating database: {e}")


## **Clean Extracted Data**

In [8]:
# === RUN THE PIPELINE ===
if __name__ == "__main__":
    print("🚀 Starting full student import pipeline...\n")
    merged_df = merge_and_tag()
    cleaned_df = clean_data(merged_df)
    update_database(cleaned_df)
    print("🎉 All done!")

🚀 Starting full student import pipeline...

✅ Cleaned data saved to D:\GITHUB\kotak-school-dbms\output_data\students_data.csv
🗑️ Old records deleted from 'students'.
⏳ Inserting data into DB...
✅ Insert complete. 📊 Table now contains 3382 records.

🎉 All done!


<h2 align="center"><b>FEE REPORTS</b></h2>

## **Google Console Service Account: myschooldb@woven-solution-446513-f2.iam.gserviceaccount.com**

## **Import Necessary Libraries & Define Global Variables**

In [9]:
# === CONFIG ===
GOOGLE_JSON_PATHS = {
    "2024-25": r"D:\GITHUB\kotak-school-dbms\google_api_keys\woven-solution-446513-f2-2024-25-fees.json",
    "2025-26": r"D:\GITHUB\kotak-school-dbms\google_api_keys\woven-solution-446513-f2-2025-26-fees.json"
}

GOOGLE_SHEET_TITLES = {
    "2024-25": "Fee Reports 2024-25",
    "2025-26": "Fee Reports 2025-26",
}

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "kotakschooldb",
}

TABLE_NAME = "fees_table"


## **Function for Fetching Data**

In [10]:
# === FETCH GOOGLE SHEET ===
def fetch_data(sheet_title, worksheet_name="Overall Sheet", json_path=None):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
    client = gspread.authorize(creds)
    spreadsheet = client.open(sheet_title)
    sheet = spreadsheet.worksheet(worksheet_name)
    data = sheet.get_all_records(head=3)
    return pd.DataFrame(data)

# === CLEAN COLUMN NAMES ===
def clean_column_names(df):
    df.columns = df.columns.str.strip()
    return df

In [11]:
# === MERGE AND TAG STATUS ===
def merge_and_tag():
    df_2024 = clean_column_names(fetch_data(GOOGLE_SHEET_TITLES["2024-25"], "Overall Sheet", GOOGLE_JSON_PATHS["2024-25"]))
    df_2025 = clean_column_names(fetch_data(GOOGLE_SHEET_TITLES["2025-26"], "Overall Sheet", GOOGLE_JSON_PATHS["2025-26"]))

    df_2024["academic_year"] = "2024-25"
    df_2025["academic_year"] = "2025-26"

    return pd.concat([df_2024, df_2025], ignore_index=True)

## **Function for Cleaning Data**

In [12]:
def clean_data(df):
    df = df[:-1][:-6]

    df.columns = ['SNo', 'STUDENT_NAME', 'ADM_NO', 'FB_NO', 'CLASS',
                  'Term1', 'Term2', 'Term3', 'Term4', 'Total_Fee_Paid',
                  'Discount_Concession', 'Total_Fee_Due', 'PermissionUpto',
                  'Fine', 'Payment_Status', 'ClassNo',"AcNo",'Concession_type', "staff_name", "academic_year"]
    
    df.columns = df.columns.str.strip().str.lower()

    # 🚫 Remove blank admission numbers
    df = df[df["adm_no"].astype(str).str.strip() != ""]
    df = df.dropna(subset=["adm_no"])
    # 🚫 Remove blank admission numbers
    df = df[df["student_name"].astype(str).str.strip() != ""]
    df = df.dropna(subset=["student_name"])

    columns_to_convert = ["term1", "term2", "term3", "term4", "total_fee_paid",
                          "discount_concession", "total_fee_due", "fine"]
    
    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce').fillna(0)

    df = df.drop(columns=["acno", 'concession_type'])

    df["sno"] = range(1, len(df) + 1)

    df = df.sort_values(by=["sno"])

    df["total_fees"] = df["total_fee_paid"] + df["discount_concession"] + df["total_fee_due"]

    df = df.sort_values(by=["academic_year", "classno", "student_name"], ascending=[True, True, True])
    
    df.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\fees_report.csv", index=False)
    
    PaymentStatus = df[["payment_status"]].drop_duplicates().reset_index(drop=True)
    PaymentStatus["payment_status_id"] = range(1, len(PaymentStatus) + 1)
    PaymentStatus = PaymentStatus[["payment_status_id", "payment_status"]]
    PaymentStatus.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\payment_status_table.csv", index=False)
    print("✅ Payment Status Table created successfully.\n")
    
    df = pd.merge(df, PaymentStatus, on="payment_status", how="left")
    
    df.drop(columns=["permissionupto","payment_status"], inplace=True)
    
    df.columns = df.columns.str.lower().str.strip()

    return df


## **Function for Updating the Database**

In [13]:
# PostgreSQL Connection Credentials
POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "kotakschooldb",
}

# Encode password for URL safety
password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])

# Create Engine
engine = create_engine(f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
                       f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}")

def table_exists(table_name):
    """Check if table exists in PostgreSQL database"""
    check_query = f"""
    SELECT EXISTS (
        SELECT FROM information_schema.tables 
        WHERE table_name = '{table_name}'
    );
    """
    with engine.connect() as conn:
        result = conn.execute(text(check_query)).scalar()
    return result

def create_table():
    """Create table only if it does not exist"""
    table_name = "fees_table"
    
    if table_exists(table_name):
        print(f"✅ Table '{table_name}' already exists.")
        return

    create_table_query = """
    CREATE TABLE fees_table (
    sno SERIAL PRIMARY KEY,
    student_name TEXT NOT NULL,
    adm_no TEXT,
    fb_no TEXT,
    class TEXT,
    term1 NUMERIC DEFAULT 0,
    term2 NUMERIC DEFAULT 0,
    term3 NUMERIC DEFAULT 0,
    term4 NUMERIC DEFAULT 0,
    total_fee_paid NUMERIC DEFAULT 0,
    discount_concession NUMERIC DEFAULT 0,
    total_fee_due NUMERIC DEFAULT 0,
    fine NUMERIC DEFAULT 0,
    classno TEXT,
    staff_name TEXT,
    academic_year TEXT NOT NULL,
    total_fees INTEGER DEFAULT 0,
    payment_status_id INTEGER
);
    """
    
    try:
        with engine.connect() as conn:
            conn.execute(text(create_table_query))
            print(f"✅ Table '{table_name}' created successfully.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")


In [14]:
import urllib.parse
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

def update_database(df):
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    engine = create_engine(
        f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
        f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}"
    )

    try:
        with engine.begin() as conn:
            # ✅ Truncate existing table and reset serial ID
            conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME} RESTART IDENTITY CASCADE;"))
            print(f"✅ All records from the '{TABLE_NAME}' table have been deleted.\n")

            # ✅ Add UNIQUE constraint on 'admissionno' (if it doesn't exist)
            conn.execute(text(f"""
                DO $$ 
                BEGIN 
                    -- Drop old constraint if exists
                    IF EXISTS (
                        SELECT 1 FROM information_schema.table_constraints 
                        WHERE table_name = '{TABLE_NAME}' AND constraint_name = 'unique_admissionno'
                    ) THEN
                        ALTER TABLE {TABLE_NAME} DROP CONSTRAINT unique_admissionno;
                    END IF;

                    -- Add new composite unique constraint if not exists
                    IF NOT EXISTS (
                        SELECT 1 FROM information_schema.table_constraints 
                        WHERE table_name = '{TABLE_NAME}' AND constraint_name = 'unique_adm_year'
                    ) THEN
                        ALTER TABLE {TABLE_NAME} ADD CONSTRAINT unique_adm_year UNIQUE ("adm_no", "academic_year");
                    END IF;
                END $$;
            """))

            print(f"✅ Unique constraint on 'admissionno' ensured in the '{TABLE_NAME}' table.\n")

        print("✅ Table cleared. Proceeding with data insertion...\n")

        # ✅ Normalize column names
        df.columns = df.columns.str.lower()

        # ✅ Drop generated or unneeded columns
        if "totalfees" in df.columns:
            df = df.drop(columns=["totalfees"])
            print("✅ 'totalfees' column removed before insertion (generated column).\n")

        # ✅ Insert data in chunks
        df.to_sql(
            name=TABLE_NAME,
            con=engine,
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )

        print(f"✅ {len(df)} records successfully inserted into '{TABLE_NAME}'.\n")

    except SQLAlchemyError as e:
        print(f"❌ An error occurred during database update: {e}")


## **Main Execution Block**

In [15]:
if __name__ == "__main__":
    # * Merge and tag both years
    combined_df = merge_and_tag()
    print("✅ Raw data merged from both years.\n")

    # * Clean and process the merged data
    cleaned_df = clean_data(combined_df)
    print("✅ Data cleaned and transformed successfully.\n")
    print("✅ Final columns are:\n", cleaned_df.columns.to_list())

    # * Create table if it does not exist
    create_table()
    print("\n✅ Table check/creation complete.\n")

    # * Drop duplicates by adm_no + year before insert
    cleaned_df = cleaned_df.drop_duplicates(subset=["adm_no", "academic_year"])
    print(f"✅ Deduplicated. Final records to upload: {len(cleaned_df)}\n")

    # * Upload data using safe insertion
    update_database(cleaned_df)


✅ Raw data merged from both years.

✅ Payment Status Table created successfully.

✅ Data cleaned and transformed successfully.

✅ Final columns are:
 ['sno', 'student_name', 'adm_no', 'fb_no', 'class', 'term1', 'term2', 'term3', 'term4', 'total_fee_paid', 'discount_concession', 'total_fee_due', 'fine', 'classno', 'staff_name', 'academic_year', 'total_fees', 'payment_status_id']
✅ Table 'fees_table' already exists.

✅ Table check/creation complete.

✅ Deduplicated. Final records to upload: 3382

✅ All records from the 'fees_table' table have been deleted.

✅ Unique constraint on 'admissionno' ensured in the 'fees_table' table.

✅ Table cleared. Proceeding with data insertion...

✅ 3382 records successfully inserted into 'fees_table'.



<h2 align="center"><b>DAY WISE REPORTS</b></h2>

## **Import Required Libraries**

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine, text

## **Define Login Credentials and MySQL Credentials**

In [17]:
# * 🔹 Login Credentials
login_url = "https://app.myskoolcom.tech/kotak_vizag/login"
urls_with_years = [
    ("https://app.myskoolcom.tech/kotak_vizag/office_fee/fee_reports_day_wise_receipt_wise_print?academic_years_id=1", "2024-25"),
    ("https://app.myskoolcom.tech/kotak_vizag/office_fee/fee_reports_day_wise_receipt_wise_print?academic_years_id=7", "2025-26"),
]

credentials = {
    "uname": "harikiran",
    "psw": "812551"
}

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "kotakschooldb",
}

TABLE_NAME = "daywise_fees_collection"

## **Define Functions for Each Step**

In [18]:
def login_to_website():
    session = requests.Session()
    login_response = session.post(login_url, data=credentials)

    if "Invalid" in login_response.text:
        print("❌ Login failed! Check credentials.\n")
        return None
    else:
        print("✅ Login successful!\n")
        return session

## **Function to Fetch Fee Report Page**

In [19]:
def fetch_fee_report_page(session, data_url):
    response = session.get(data_url)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table")
    return table

## **Function to Extract Data from Table**

In [20]:
def extract_data_from_table(table):
    rows = []
    for tr in table.find_all("tr"):
        cols = [td.text.strip() for td in tr.find_all("td")]
        if cols:
            rows.append(cols)

    if not rows:
        raise ValueError("No rows found in table")

    num_cols = len(rows[0])

    if num_cols == 12:
        header_row = [
            "SNo", "RecieptNo", "Class", "AdmissionNo", "StudentName", 
            "Date", "-", "Abacus / Vediic Maths", "TERM FEE", "TERM FEE2", 
            "ReceivedAmount", "Remarks"
        ]
    elif num_cols == 11:
        header_row = [
            "SNo", "RecieptNo", "Class", "AdmissionNo", "StudentName", 
            "Date", "-", "Abacus / Vediic Maths", "TERM FEE", 
            "ReceivedAmount", "Remarks"
        ]
    else:
        raise ValueError(f"⚠️ Unexpected number of columns: {num_cols}")

    df = pd.DataFrame(rows, columns=header_row)
    return df


## **Function to Clean Data**

In [21]:
def clean_and_tag_data(df, academic_year):
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
    df['AdmissionNo'] = df['AdmissionNo'].astype(str)
    df["Class"] = df["Class"].str.replace("/", " - ")
    df["ReceivedAmount"] = pd.to_numeric(df["ReceivedAmount"], errors="coerce")
    df["academic_year"] = academic_year  # Add academic year tag

    # Apply "TERM" row slicing only for 2024-25
    if academic_year == "2024-25":
        term_index = df[df["SNo"].str.contains("TERM", na=False)].index
        if not term_index.empty:
            df = df.iloc[:term_index[0]]

    # Safely drop any of these columns only if they exist
    cols_to_drop = ["-", "Abacus / Vediic Maths", "TERM FEE", "TERM FEE2"]

    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns], errors="ignore")

    df = df.dropna(subset=["Date" ])

    df

    return df


## **Function to Update Database**

In [22]:
def update_database(df):
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    engine = create_engine(
        f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
        f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}"
    )

    # Define SQL table creation (only if not exists)
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
        "SNo" TEXT,
        "RecieptNo" TEXT,
        "Class" TEXT,
        "AdmissionNo" TEXT,
        "StudentName" TEXT,
        "Date" DATE,
        "ReceivedAmount" NUMERIC,
        "Remarks" TEXT,
        "academic_year" TEXT
    );
    """

    try:
        with engine.connect() as conn:
            conn.execute(text(create_table_sql))  # ✅ Ensure table exists

            conn.execute(text("BEGIN;"))
            conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME};"))
            conn.execute(text("COMMIT;"))
            print(f"✅ Table '{TABLE_NAME}' ensured and truncated.\n")

            # ✅ Insert DataFrame
            df.to_sql(name=TABLE_NAME, con=engine, if_exists='append', index=False, method="multi", chunksize=1000)
            print(f"✅ {len(df)} records inserted into '{TABLE_NAME}' successfully.\n")
    except Exception as e:
        print(f"⚠️ Error inserting data: {e}")


In [23]:
def main():
    session = login_to_website()
    if session is None:
        return

    all_dfs = []

    for url, year in urls_with_years:
        print(f"📄 Fetching data for academic year: {year}")
        table = fetch_fee_report_page(session, url)

        if table:
            df = extract_data_from_table(table)
            print(df.info())
            
            df = clean_and_tag_data(df, year)
            print(f"✅ Data extracted for year {year} with {len(df)} records.\n")

                        
            print(df.info())
            print(df.head())
            df.to_csv(fr"D:\GITHUB\kotak-school-dbms\output_data\fee_collection_{year}.csv", index=False)
            all_dfs.append(df)
        else:
            print(f"❌ Table not found for year {year}!")

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        final_df.to_csv(r"D:\GITHUB\kotak-school-dbms\output_data\merged_fee_collection.csv", index=False)
        print("✅ Data saved to merged_fee_collection.csv\n")

        update_database(final_df)
        print("✅ Columns:\n", final_df.columns)
        print(f"✅ Total {len(final_df)} records entered into database.")
    else:
        print("❌ No data to process!")

## **Run the Main Function**

In [24]:
# * Run the main function
main()


✅ Login successful!

📄 Fetching data for academic year: 2024-25
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6618 entries, 0 to 6617
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   SNo                    6618 non-null   object
 1   RecieptNo              6618 non-null   object
 2   Class                  6614 non-null   object
 3   AdmissionNo            6614 non-null   object
 4   StudentName            6614 non-null   object
 5   Date                   6614 non-null   object
 6   -                      6614 non-null   object
 7   Abacus / Vediic Maths  6358 non-null   object
 8   TERM FEE               6358 non-null   object
 9   TERM FEE2              6358 non-null   object
 10  ReceivedAmount         5816 non-null   object
 11  Remarks                5816 non-null   object
dtypes: object(12)
memory usage: 620.6+ KB
None
✅ Data extracted for year 2024-25 with 5816 records.

<class 'panda

<h2 align="center"><b>ATTENDANCE REPORT</b></h2>

## **Import Libraries**

In [25]:
import os
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

## **Login to Website**

In [26]:
# ✅ Config
login_url = "https://app.myskoolcom.tech/kotak_vizag/login"
attendance_url = "https://app.myskoolcom.tech/kotak_vizag/admin/attedance_grid"

credentials = {
    "uname": "harikiran",
    "psw": "812551"
}

download_folder = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports"
merged_output_path = os.path.join(download_folder, "MergedAttendance_2025_26.csv")

academic_ranges = {
    "2025-26": ("2025-06-16", datetime.today().strftime("%Y-%m-%d"))
    # "2025-26": ("2025-06-16", datetime.today().strftime("%Y-%m-%d"))
}

In [27]:
# ✅ Setup Chrome Driver
chrome_options = webdriver.ChromeOptions()
prefs = {"download.default_directory": download_folder}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
wait = WebDriverWait(driver, 10)

# ✅ Functions
def login():
    driver.get(login_url)
    wait.until(EC.presence_of_element_located((By.NAME, "uname"))).send_keys(credentials["uname"])
    driver.find_element(By.NAME, "psw").send_keys(credentials["psw"])
    driver.find_element(By.NAME, "psw").send_keys(Keys.RETURN)
    print("✅ Logged in successfully!")
    time.sleep(5)

def set_date_range(start, end):
    driver.get(attendance_url)
    time.sleep(2)
    from_date_input = wait.until(EC.presence_of_element_located((By.ID, "from_attendance_date")))
    driver.execute_script("arguments[0].removeAttribute('readonly')", from_date_input)
    from_date_input.clear()
    from_date_input.send_keys(start)

    to_date_input = wait.until(EC.presence_of_element_located((By.ID, "to_attendance_date")))
    driver.execute_script("arguments[0].removeAttribute('readonly')", to_date_input)
    to_date_input.clear()
    to_date_input.send_keys(end)

    print(f"✅ Date range set: {start} to {end}")

def download_csv(filename):
    try:
        if os.path.exists(filename):
            os.remove(filename)
        download_button = wait.until(EC.element_to_be_clickable((By.ID, "smaplecsv")))
        download_button.click()
        time.sleep(8)
        downloaded = sorted(
            [f for f in os.listdir(download_folder) if f.endswith(".csv")],
            key=lambda x: os.path.getctime(os.path.join(download_folder, x)),
            reverse=True
        )[0]
        os.rename(os.path.join(download_folder, downloaded), filename)
        print(f"✅ Downloaded and renamed to: {filename}")
    except Exception as e:
        print(f"❌ Error downloading file: {e}")

def date_batches(start, end, months=1):
    start_date = datetime.strptime(start, "%Y-%m-%d")
    end_date = datetime.strptime(end, "%Y-%m-%d")
    while start_date < end_date:
        batch_end = min(start_date + timedelta(days=30 * months), end_date)
        yield (start_date.strftime("%Y-%m-%d"), batch_end.strftime("%Y-%m-%d"))
        start_date = batch_end + timedelta(days=1)

def merge_csvs(folder, output_file, year_filter="2025-26"):
    all_csvs = [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if f.endswith(".csv") and year_filter in f
    ]

    merged_df = pd.DataFrame()

    for f in all_csvs:
        try:
            df = pd.read_csv(f, low_memory=False)
            if "Students Number" in df.columns:
                # Merge logic: remove duplicates by date + student number
                df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
                df = df.dropna(subset=["Date", "Students Number"])

                # Merge with deduplication
                if not merged_df.empty:
                    merged_df = pd.concat([merged_df, df], ignore_index=True)
                    merged_df.drop_duplicates(subset=["Date", "Students Number"], keep="last", inplace=True)
                else:
                    merged_df = df
                print(f"🔄 Merged file (with Students Number): {os.path.basename(f)}")
            else:
                # Append directly if "Students Number" not found
                merged_df = pd.concat([merged_df, df], ignore_index=True)
                print(f"➕ Appended file (no Students Number): {os.path.basename(f)}")
        except Exception as e:
            print(f"❌ Error reading file {f}: {e}")

    merged_df.to_csv(output_file, index=False)
    print(f"✅ Final merged file saved: {output_file}")

    # ✅ MAIN Execution
login()

# ✅ MAIN Execution
login()

for year, (start, end) in academic_ranges.items():
    print(f"\n📅 Downloading attendance for {year}")
    for i, (s, e) in enumerate(date_batches(start, end)):
        s_fmt = datetime.strptime(s, "%Y-%m-%d")
        e_fmt = datetime.strptime(e, "%Y-%m-%d")
        filename = f"Attendance_{year}_{s_fmt.strftime('%b')}_{e_fmt.strftime('%b')}.csv"
        filepath = os.path.join(download_folder, filename)
        set_date_range(s_fmt.strftime("%Y-%m-%d"), e_fmt.strftime("%Y-%m-%d"))
        download_csv(filepath)

# ❌ No merging now – only individual files will be downloaded and renamed
# merge_csvs(download_folder, merged_output_path, year_filter="2025-26")

driver.quit()
print("✅ All attendance downloads complete – individual files saved!")


✅ Logged in successfully!
✅ Logged in successfully!

📅 Downloading attendance for 2025-26
✅ Date range set: 2025-06-16 to 2025-07-16
✅ Downloaded and renamed to: D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\Attendance_2025-26_Jun_Jul.csv
✅ Date range set: 2025-07-17 to 2025-07-27
✅ Downloaded and renamed to: D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\Attendance_2025-26_Jul_Jul.csv
✅ All attendance downloads complete – individual files saved!


## **📌 Step 1: Import Libraries**

In [83]:
import pandas as pd
from sqlalchemy import create_engine
import logging
import numpy as np
import urllib
import logging
import traceback
from datetime import datetime

# * Configure logging
logging.basicConfig(filename=r"D:\GITHUB\kotak-school-dbms\output_data\attendance_report.log", level=logging.ERROR, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


## **📌 Step 2: Define PostgreSQl Credentials & Table Name**

In [90]:
# * MySQL Credentials
POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "kotakschooldb",
}
TABLE_NAME = "attendance_report"


## **📌 Step 3: Load and Clean Data**

In [91]:
def load_and_clean_data(file1, file2, file3, file4=None, file5=None):
    # Load DataFrames
    dfs = [pd.read_csv(f) for f in [file1, file2, file3, file4, file5] if f is not None]

    # Clean column names
    for i in range(len(dfs)):
        dfs[i].columns = dfs[i].columns.str.strip().str.replace('"', '', regex=False)

    # Ensure unique columns before merge to avoid suffix collision
    base_df = dfs[0]

    for df in dfs[1:]:
        # Remove columns from df that would cause _x/_y clashes (except 'Students Number')
        conflict_cols = [col for col in df.columns if col in base_df.columns and col != 'Students Number']
        df = df.drop(columns=conflict_cols, errors='ignore')

        # Merge safely
        base_df = base_df.merge(df, on="Students Number", how="outer")

    df = base_df

    # Merge fields like Name, Class if duplicated from earlier columns
    for field in ['Name', 'Class']:
        col_x, col_y = f"{field}_x", f"{field}_y"
        if col_x in df.columns and col_y in df.columns:
            df[field] = df[col_x].combine_first(df[col_y])
            df.drop([col_x, col_y], axis=1, inplace=True)
        elif col_x in df.columns:
            df[field] = df.pop(col_x)
        elif col_y in df.columns:
            df[field] = df.pop(col_y)

    # Drop any remaining _x/_y suffix columns
    df = df.drop(columns=[col for col in df.columns if col.endswith('_x') or col.endswith('_y')], errors='ignore')

    # Rename key identifier
    df = df.rename(columns={"Students Number": "AdmissionNo"})

    # Drop unnecessary columns
    drop_cols = ['Present Days', 'Absent Days', 'Toral Working Days']
    df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

    # Reorder columns
    key_cols = ['AdmissionNo', 'Name', 'Class']
    other_cols = [col for col in df.columns if col not in key_cols]
    df = df[key_cols + other_cols]

    return df


# **📌 Step 4: Process Attendance Data**

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd

def process_attendance_data(df):
    # * Step 1: Clean 'AdmissionNo'
    df = df[~(df["AdmissionNo"].astype(str) == "786") & ~df["AdmissionNo"].astype(str).str.match(r"^[a-zA-Z]")].copy()

    # * Step 2: Extract Class and Section
    df["Class"] = df["Class"].astype(str).str.replace(r"ICSE \((.*?)\)", r"\1", regex=True)

    # * Load students with academic_year info
    student_df = pd.read_csv(r"D:\GITHUB\kotak-school-dbms\output_data\fees_report.csv")[["adm_no", "academic_year"]]
    print("✅ Students Before Merging\n", len(df["AdmissionNo"].unique()))

    # * Keep only students of academic_year 2024-25
    student_df_2024_25 = student_df[student_df["academic_year"] == "2024-25"]
    student_df_2025_26 = student_df[student_df["academic_year"] == "2025-26"]

    df = df[df["AdmissionNo"].isin(student_df["adm_no"])]
    print("✅ Students After Merging\n", len(df["AdmissionNo"].unique()))

    # * Step 3: Unpivot DataFrame
    df_unpivot = pd.melt(df, id_vars=["AdmissionNo", "Name", "Class"], var_name="Date", value_name="AttendanceStatus")
    df_unpivot["Date"] = pd.to_datetime(df_unpivot["Date"], format='%d.%m.%Y', errors='coerce')
    df_unpivot["id"] = range(1, len(df_unpivot) + 1)

    if df_unpivot["Date"].isna().sum() > 0:
        print("⚠️ Warning: Some Date values were invalid and converted to NaT.")

    df_unpivot = df_unpivot.sort_values("Date", ascending=False).reset_index(drop=True)

    first_attendance_dates = df_unpivot[df_unpivot['AttendanceStatus'].notna()].groupby('AdmissionNo')['Date'].min()
    df_unpivot['AttendanceStatus'] = df_unpivot.apply(
        lambda row: "Not Joined" if row['Date'] < first_attendance_dates.get(row['AdmissionNo'], row['Date']) else row['AttendanceStatus'],
        axis=1
    )

    priority_map = {'P': 2, 'A': 1, 'H': 3, 'Not Joined': 4, 'TC': 5}
    df_unpivot['Priority'] = df_unpivot["AttendanceStatus"].map(priority_map)
    df_unpivot = df_unpivot.sort_values(by=['AdmissionNo', 'Date', 'Priority']) \
                           .drop_duplicates(subset=['AdmissionNo', 'Date'], keep='first') \
                           .drop(columns=['Priority'])

    df_unpivot = df_unpivot[["id", 'Date', 'AdmissionNo', 'Name', 'Class', 'AttendanceStatus']]
    df_unpivot.sort_values(by=['Date'], ascending=False, inplace=True)
    df_unpivot['Class'] = df_unpivot['Class'].str.replace("Pre KG - ", "Pre KG")

    df_unpivot["AttendanceStatus"] = df_unpivot["AttendanceStatus"].replace({
        'P': "Present", 'A': "Absent", 'H': "Holiday"
    })

    # * Step X: Add Academic Year
    df_unpivot['academic_year'] = df_unpivot['Date'].apply(
        lambda d: "2024-25" if pd.Timestamp("2024-07-17") <= d <= pd.Timestamp("2025-03-31")
        else "2025-26" if pd.Timestamp("2025-06-16") <= d <= pd.Timestamp(datetime.today().date())
        else ""
    )

    # * Step X: Class & Section Mappings
    class_mapping_2024_25 = {
        "Pre KG": 1, "LKG - A": 2, "LKG - B": 3, "UKG - A": 4, "UKG - B": 5, "UKG - C": 6,
        "I - A": 7, "I - B": 8, "I - C": 9, "I - D": 10, "II - A": 11, "II - B": 12, "II - C": 13, "II - D": 14,
        "III - A": 15, "III - B": 16, "III - C": 17, "III - D": 18, "IV - A": 19, "IV - B": 20, "IV - C": 21, "IV - D": 22,
        "V - A": 23, "V - B": 24, "V - C": 25, "V - D": 26, "VI - A": 27, "VI - B": 28, "VI - C": 29, "VI - D": 30,
        "VII - A": 31, "VII - B": 32, "VII - C": 33, "VII - D": 34, "VIII - A": 35, "VIII - B": 36, "VIII - C": 37,
        "IX - A": 38, "IX - B": 39, "IX - C": 40, "X - A": 41, "X - B": 42, "X - C": 43
    }

    class_mapping_2025_26 = {
        "Pre KG": 1, "LKG - A": 2, "LKG - B": 3, "UKG - A": 4, "UKG - B": 5, "UKG - C": 6,
        "I - A": 7, "I - B": 8, "I - C": 9,  # no I - D
        "II - A": 10, "II - B": 11, "II - C": 12, "II - D": 13,
        "III - A": 14, "III - B": 15, "III - C": 16, "III - D": 17,
        "IV - A": 18, "IV - B": 19, "IV - C": 20, "IV - D": 21,
        "V - A": 22, "V - B": 23, "V - C": 24, "V - D": 25,
        "VI - A": 26, "VI - B": 27, "VI - C": 28, "VI - D": 29,
        "VII - A": 30, "VII - B": 31, "VII - C": 32, "VII - D": 33,
        "VIII - A": 34, "VIII - B": 35, "VIII - C": 36, "VIII - D": 37,
        "IX - A": 38, "IX - B": 39, "IX - C": 40,
        "X - A": 41, "X - B": 42, "X - C": 43
    }

    # * Apply correct mapping based on academic year
    def map_class_no(row):
        if row['academic_year'] == '2024-25':
            return class_mapping_2024_25.get(row['Class'], np.nan)
        elif row['academic_year'] == '2025-26':
            return class_mapping_2025_26.get(row['Class'], np.nan)
        return np.nan

    df_unpivot['ClassNo'] = df_unpivot.apply(map_class_no, axis=1)

    print("✅ ClassNo assigned based on academic year and Class.")
    print(df_unpivot[df_unpivot[['Class', 'ClassNo', 'academic_year']].isna()])

    # * Step 14–17: Same as before...
    grade_mapping = [
        ("Pre KG", 1), ("LKG", 2), ("UKG", 3),
        ("I", 4), ("II", 5), ("III", 6), ("IV", 7), ("V", 8),
        ("VI", 9), ("VII", 10), ("VIII", 11), ("IX", 12), ("X", 13)
    ]
    conditions = [df_unpivot['Class'].str.contains(fr"\b{k}\b", na=False, regex=True) for k, _ in grade_mapping]
    choices = [v for _, v in grade_mapping]
    df_unpivot['classId'] = np.select(conditions, choices, default=0)

    AttendanceStatus_mapping = [("Absent", 1), ("Present", 2), ("Not Joined", 3), ("Holiday", 4)]
    conditions = [df_unpivot['AttendanceStatus'].str.contains(k, na=False) for k, _ in AttendanceStatus_mapping]
    choices = [v for _, v in AttendanceStatus_mapping]
    df_unpivot['AttendanceStatusId'] = np.select(conditions, choices, default=0)

    branch_mapping = [
        ('Pre KG', 1), ('LKG', 1), ('UKG', 1),
        ('I', 2), ('II', 2), ('III', 2), ('IV', 2), ('V', 2),
        ('VI', 3), ('VII', 3), ('VIII', 3), ('IX', 3), ('X', 3)
    ]
    conditions = [df_unpivot['Class'].str.contains(fr"\b{k}\b", na=False, regex=True) for k, _ in branch_mapping]
    choices = [v for _, v in branch_mapping]
    df_unpivot['branchId'] = np.select(conditions, choices, default=0)

    # ✅ Final output: lowercase and return
    df_unpivot = df_unpivot[["id", 'Date', 'AdmissionNo', 'ClassNo', 'classId', 'branchId', 'AttendanceStatusId', 'academic_year']]
    df_unpivot.columns = [c.lower() for c in df_unpivot.columns]
    return df_unpivot


## **📌 Step 5: Insert Data into PostgreSQL**

In [93]:
from sqlalchemy import text

def ensure_table_exists():
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
        id SERIAL PRIMARY KEY,
        date DATE,
        admissionno TEXT,
        attendancestatusid INTEGER,
        academic_year TEXT NOT NULL
    );
    """
    try:
        with engine.begin() as connection:  # ✅ ensures DDL is committed
            connection.execute(text(create_table_sql))
        print(f"✅ Table '{TABLE_NAME}' ensured.")
    except Exception as e:
        print(f"❌ Failed to create or check table '{TABLE_NAME}': {e}")


In [94]:
# Create database engine
password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
engine = create_engine(
    f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
    f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}"
)

def update_database(df):
    """Use PostgreSQL COPY for ultra-fast data insertion."""
    csv_path = (r"D:\GITHUB\kotak-school-dbms\output_data\attendance_report.csv")

    # ✅ Ensure column names are lowercase to match table definition
    df.columns = [c.lower() for c in df.columns]

    # ✅ Save DataFrame to CSV
    df.to_csv(csv_path, index=False, header=False)

    try:
        conn = engine.raw_connection()
        cursor = conn.cursor()

        print(f"🔄 Truncating table: {TABLE_NAME}")
        cursor.execute(f"TRUNCATE TABLE {TABLE_NAME};")
        conn.commit()

        with open(csv_path, "r") as f:
            cursor.copy_from(f, TABLE_NAME, sep=",")  # ✅ lowercase and unquoted

        conn.commit()
        cursor.close()
        conn.close()

        print(f"✅ Data copied to '{TABLE_NAME}' using COPY command!")

    except Exception as e:
        print(f"❌ COPY failed: {e}")
        logging.error(traceback.format_exc())


## **📌 Step 6: Run the Full Pipeline**

In [95]:
def main():
    file1 = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\AttendanceReportUptoSeptember_2024_25.csv"
    file2 = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\AttendanceOctoberToDecember_2024_25.csv"
    file3 = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\AttendanceUptoMarch_2024_25.csv"
    file4 = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\Attendance_2025-26_Jun_Jul.csv"
    file5 = r"D:\GITHUB\kotak-school-dbms\source_data\Attendance Reports\Attendance_2025-26_Jun_Jul.csv"
    output_file = r"D:\GITHUB\kotak-school-dbms\output_data\attendance_report.csv"
    
    try:
        print("Loading and cleaning data...\n")
        df = load_and_clean_data(file1, file2, file3, file4, file5)
        print(f"✅ Data loaded with {df.shape[0]} rows.\n")
        
        print("Processing attendance data...\n")
        df_unpivot = process_attendance_data(df)
        df_unpivot.to_csv(output_file, index=False)
        print(f"✅ Processed data with {df_unpivot.shape[0]} rows.\n")
        print("✅ Columns are:\n", df_unpivot.columns)
        print(max(df_unpivot["date"]))
        print((df_unpivot.head()))
                
        print("Updating database...\n")
        ensure_table_exists()
        update_database(df_unpivot)
        print("✅ Data updated successfully!\n")

        print("✅ Attendance report processing completed successfully!\n")
        print(f"✅ No of Rows: {df_unpivot.shape[0]}\n")
        
    except Exception as e:
        print(f"❌ An unexpected error occurred. Error: {e}\n")
        logging.error(f"❌ Unexpected error: {e}\n")


# * Run the script
main()


Loading and cleaning data...

✅ Data loaded with 1989 rows.

Processing attendance data...

✅ Students Before Merging
 1969
✅ Students After Merging
 1939
✅ ClassNo assigned based on academic year and Class.
✅ Processed data with 560371 rows.

✅ Columns are:
 Index(['id', 'date', 'admissionno', 'attendancestatusid', 'academic_year'], dtype='object')
2025-07-16 00:00:00
          id       date admissionno  attendancestatusid academic_year
0     560371 2025-07-16       17371                   2       2025-26
256   559324 2025-07-16       16162                   2       2025-26
1363  560188 2025-07-16       17188                   2       2025-26
580   558615 2025-07-16       14510                   1       2025-26
230   558995 2025-07-16       15523                   2       2025-26
Updating database...

✅ Table 'attendance_report' ensured.
🔄 Truncating table: attendance_report
✅ Data copied to 'attendance_report' using COPY command!
✅ Data updated successfully!

✅ Attendance report proc

<h2 align="center"><b>Class Table</b></h2>

In [35]:
import pandas as pd
from sqlalchemy import create_engine, text

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "kotakschooldb",
}
TABLE_NAME = "class_table"

In [ ]:
df = pd.read_csv(r"D:\GITHUB\kotak-school-dbms\output_data\class_section_grade_table.csv")
df.head()

,ClassNo,Class,classId,className,branchId,branchName
0,1,Pre KG,1,Pre KG,1,Kindergarten
1,2,LKG - A,2,LKG,1,Kindergarten
2,3,LKG - B,2,LKG,1,Kindergarten
3,4,UKG - A,3,UKG,1,Kindergarten
4,5,UKG - B,3,UKG,1,Kindergarten


In [ ]:
df.columns

Index(['ClassNo', 'Class', 'classId', 'className', 'branchId', 'branchName'], dtype='object')

In [ ]:
import time
import traceback
import logging
import pandas as pd
import urllib
import io
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

# Retry settings
MAX_RETRIES = 3
RETRY_DELAY = 5  # Seconds

def bulk_insert_postgres(df, conn, table_name):
    """Fast bulk insert using PostgreSQL COPY command."""
    with conn.connection.cursor() as cur:
        output = io.StringIO()
        df.to_csv(output, sep="\t", index=False, header=False)
        output.seek(0)
        cur.copy_from(output, table_name, sep="\t", null="NULL")
        conn.connection.commit()

def update_database(df):
    """Insert attendance data into PostgreSQL database with retry logic."""
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    engine = create_engine(f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
                           f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}")

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f"🔄 Attempt {attempt}: Connecting to database {POSTGRES_CREDENTIALS['database']} at {POSTGRES_CREDENTIALS['host']}...")
            with engine.begin() as conn:
                print(f"✅ Connection established.")

                # Create Table if it does not exist
                print(f"Checking if table '{TABLE_NAME}' exists...")
                
                # Truncate the table before inserting data
                print(f"Truncating existing table: {TABLE_NAME}")
                conn.execute(text(f"TRUNCATE TABLE {TABLE_NAME} CASCADE;"))
                
                print(f"Deleting data from {TABLE_NAME} table...")
                conn.execute(text(f"DELETE FROM {TABLE_NAME};"))


                # Fast Bulk Insert
                print(f"Inserting data into {TABLE_NAME} table...")
                bulk_insert_postgres(df, conn, TABLE_NAME)

                print(f"✅ Data successfully inserted into '{TABLE_NAME}' table.")
                return  # Exit function if successful

        except OperationalError as e:
            print(f"❌ OperationalError: {e}")
            logging.error(f"❌ OperationalError: {e}")
            logging.error("Error Traceback:\n" + traceback.format_exc())

            if attempt < MAX_RETRIES:
                print(f"🔄 Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                print("❌ Max retries reached. Could not update the database.")
                logging.error("❌ Max retries reached. Could not update the database.")
                return


In [ ]:
update_database(df)

🔄 Attempt 1: Connecting to database schooldb at localhost...
✅ Connection established.
Checking if table 'class_table_2024_25' exists...
Truncating existing table: class_table_2024_25
Deleting data from class_table_2024_25 table...
Inserting data into class_table_2024_25 table...
✅ Data successfully inserted into 'class_table_2024_25' table.


<h2 align="center"><b>FEE COLLECTION REPORT 2024-25</b></h2>

## **Import Required Libraries**

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import numpy as np
import logging
from sqlalchemy import create_engine, text

In [8]:
# 📌 Logging
logging.basicConfig(filename="fee_collection_merge.log", level=logging.ERROR)

# 🔐 Credentials & URLs
login_url = "https://app.myskoolcom.tech/kotak_vizag/login"
credentials = {"uname": "harikiran", "psw": "812551"}
urls = {
    "2024_25": "https://app.myskoolcom.tech/kotak_vizag/office_fee/fee_consolidate_report_print?&from=2024-04-01&academic_years_id=7&status=1&imageField=Search",
    "2025_26": "https://app.myskoolcom.tech/kotak_vizag/office_fee/fee_consolidate_report_print?&from=2025-04-01&academic_years_id=1&status=1&imageField=Search"
}

# 🛠️ PostgreSQL Config
POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "kotakschooldb",
}
TABLE_NAME = "fees_collection"

In [9]:
# 🔌 Create Engine
def get_engine():
    password = urllib.parse.quote(POSTGRES_CREDENTIALS["password"])
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_CREDENTIALS['username']}:{password}"
        f"@{POSTGRES_CREDENTIALS['host']}:{POSTGRES_CREDENTIALS['port']}/{POSTGRES_CREDENTIALS['database']}"
    )

# 🔑 Login
def login_to_website():
    session = requests.Session()
    response = session.post(login_url, data=credentials)
    if "Invalid" in response.text:
        print("❌ Login failed!")
        return None
    print("✅ Login successful!")
    return session


In [21]:
# 🧾 Convert HTML table to DataFrame
def table_to_dataframe(table):
    headers = [th.get_text(strip=True) for th in table.find_all("th")]
    rows = [[td.get_text(strip=True) for td in tr.find_all("td")] for tr in table.find_all("tr")[1:]]
    return pd.DataFrame(rows, columns=headers) if rows else None

# 📥 Fetch fee table from a given URL
def fetch_fee_table(session, url):
    response = session.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("table", class_="b-t")
    all_data = []

    for table in tables:
        df = table_to_dataframe(table)
        if df is not None:
            all_data.append(df)

    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()


# 🧹 Clean data
def clean_data(df, academic_year):
    df = df[~df.iloc[:, 0].astype(str).str.startswith("Total", na=False)].copy()
    df["Admin No."] = df["Admin No."].astype(str)
    
    df.columns = ['SNo', 'AdmissionNo', 'Name', 'Abacus1', 'TermFee1',
                  'Total_Fees', 'Abacus2', 'TermFee2',
                  'Total_Fee_Paid', 'Discount_Concession', 'Total_Due']
    
    numeric_columns = ["Total_Fees", "Total_Fee_Paid", "Discount_Concession", "Total_Due"]
    for col in numeric_columns:
        df[col] = df[col].astype(str).str.replace(",", "").replace(["", "None", "nan", "NaN", np.nan], 0)
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    df = df.drop(columns=["SNo", "Abacus1", "Abacus2", "TermFee1", "TermFee2"])
    df["academic_year"] = academic_year  # Add source year column
    return df

In [22]:
from sqlalchemy import text

def ensure_fees_collection_table(engine):
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS fees_collection (
        id SERIAL PRIMARY KEY,
        admissionno TEXT,
        name TEXT,
        total_fees INTEGER,
        total_fee_paid INTEGER,
        discount_concession INTEGER,
        total_due INTEGER,
        academic_year TEXT
    );
    """
    try:
        with engine.begin() as conn:
            conn.execute(text(create_table_sql))
        print("✅ Table 'fees_collection' ensured.")
    except Exception as e:
        print(f"❌ Error creating table: {e}")


In [23]:
# 🛢️ Insert into PostgreSQL
def update_database(df, table_name):
    engine = get_engine()
    try:
        with engine.begin() as conn:
            print(f"⚠️ Deleting old records from '{table_name}'...")
            conn.execute(text(f"DELETE FROM {table_name};"))
            print(f"✅ Table '{table_name}' cleared.")
        df.columns = df.columns.str.lower()
        print(f"📥 Inserting {len(df)} rows...")
        df.to_sql(name=table_name, con=engine, if_exists='append', index=False, method='multi', chunksize=1000)
        print(f"✅ Inserted into '{table_name}' successfully.")
    except Exception as e:
        print(f"❌ Error inserting: {e}")
        logging.error(f"Database insert error: {e}")
    finally:
        engine.dispose()

In [24]:
# 🚀 Main Logic
def main():
    session = login_to_website()
    if session is None:
        return

    merged_df = pd.DataFrame()
    
    for year, url in urls.items():
        print(f"\n🔄 Fetching data for {year}...")
        raw_df = fetch_fee_table(session, url)
        if raw_df.empty:
            print(f"❌ No data for {year}!")
            continue
        clean_df = clean_data(raw_df, academic_year=year)
        merged_df = pd.concat([merged_df, clean_df], ignore_index=True)

    if merged_df.empty:
        print("❌ No data collected from any year!")
        return

    # Save CSV (optional)
    merged_df.to_csv("merged_fee_collection.csv", index=False)
    print("📁 Saved to merged_fee_collection.csv")

    # Ensure table exists
    engine = get_engine()
    ensure_fees_collection_table(engine)
    print("✅ Fees collection table ensured.")
    # Push to DB
    update_database(merged_df, TABLE_NAME)
    print(f"✅ All done! Total records: {len(merged_df)}")

if __name__ == "__main__":
    main()

✅ Login successful!

🔄 Fetching data for 2024_25...

🔄 Fetching data for 2025_26...
📁 Saved to merged_fee_collection.csv
✅ Table 'fees_collection' ensured.
✅ Fees collection table ensured.
⚠️ Deleting old records from 'fees_collection'...
✅ Table 'fees_collection' cleared.
📥 Inserting 3228 rows...
✅ Inserted into 'fees_collection' successfully.
✅ All done! Total records: 3228


<h2 align="center"><b>FEE CONCESSION REPORT 2024-25</b></h2>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
from sqlalchemy import create_engine, MetaData
from sqlalchemy.dialects.postgresql import insert
from datetime import date


## **Define Login Credentials and MySQL Credentials*
login_url = "https://app.myskoolcom.tech/kotak_vizag/login"
data_url = "https://app.myskoolcom.tech/kotak_vizag/office_fee/fee_discounts_report_receipt_wise_print?&academic_years_id=1"

credentials = {
    "uname": "harikiran",
    "psw": "812551"
}

POSTGRES_CREDENTIALS = {
    "username": "postgres",
    "password": "Hari@123",
    "host": "localhost",
    "port": "5432",
    "database": "kotakschooldb",
}

TABLE_NAME = "fee_concession_2024_25"

In [ ]:
## **Function to Log in to Website*
def login_to_website():
    session = requests.Session()
    login_response = session.post(login_url, data=credentials)
    
    if login_response.status_code != 200:
        print("❌ Login request failed! Server error.\n")
        return None
    
    soup = BeautifulSoup(login_response.text, "html.parser")
    if soup.find("div", class_="alert-danger"):
        print("❌ Login failed! Check credentials.\n")
        return None
    
    print("✅ Login successful!\n")
    return session

In [ ]:
## **Function to Fetch All Fee Tables*
def fetch_all_concession_tables(session):
    response = session.get(data_url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    concession_tables = soup.find_all("table", class_="table_view")
    if not concession_tables:
        print("❌ No fee tables found! The page structure may have changed.")
        return None

    all_data = []
    for table in concession_tables:
        df = table_to_dataframe(table)
        if df is not None:
            all_data.append(df)

    if not all_data:
        print("❌ No data extracted from tables.")
        return None

    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

In [ ]:
## **Function to Convert HTML Table to DataFrame*
def table_to_dataframe(table):
    headers = [th.get_text(strip=True) for th in table.find_all("th")]
    if len(headers) > 8:
        headers = headers[:8]  # * Keep only the first 8 columns
    
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]
        if len(cells) >= 8:
            rows.append(cells[:8])  # * Keep only the first 8 columns
    
    df = pd.DataFrame(rows, columns=headers) if rows else None
    return df

In [ ]:
import pandas as pd

def clean_data(df):
    # Standardize column names: lowercase, replace spaces with underscores
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
    
    # Drop rows where 'student_number' is missing (ensuring data consistency)
    df = df.dropna(subset=["student_number"])
    df["student_number"] = df["student_number"].astype(str).str.strip()

    # Convert 'discount_given' to numeric, handling errors gracefully
    df["discount_given"] = pd.to_numeric(df["discount_given"], errors="coerce").fillna(0.00)

    # Drop extra unnecessary columns if they exist
    df.drop(columns=['receipt_no', 'fee_name', 'fee_amount', 'total_due_amount'], errors="ignore", inplace=True)
    
    df["date"] = df["date"].astype(str).str.strip()  # Remove spaces

    # Ensure 'date' is in correct format & drop invalid entries
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
    df = df.dropna(subset=["date"])  # Drop rows with invalid dates
    
    # Add an ID column (Auto-incremental)
    df["id"] = range(1, len(df) + 1)

    # Reorder columns for consistency
    df = df[['id', 'date', 'student_number', 'student_name', 'discount_given']]

    # Reset index (ensures DataFrame integrity)
    df.reset_index(drop=True, inplace=True)

    return df


In [ ]:
import logging
import urllib.parse
import pandas as pd
from sqlalchemy import create_engine, text

def update_database(df: pd.DataFrame, table_name: str, postgres_credentials: dict):
    """Insert fee collection data into PostgreSQL efficiently, fixing column name issues."""

    # Encode password to handle special characters safely
    password = urllib.parse.quote(postgres_credentials["password"])

    # Create PostgreSQL connection engine
    engine = create_engine(
        f"postgresql+psycopg2://{postgres_credentials['username']}:{password}"
        f"@{postgres_credentials['host']}:{postgres_credentials['port']}/{postgres_credentials['database']}"
    )

    try:
        with engine.begin() as conn:  # Auto-commits or rollbacks if an error occurs
            print(f"🔄 Connecting to database {postgres_credentials['database']} at {postgres_credentials['host']}...")

            # ✅ Standardize column names to lowercase
            df.columns = df.columns.str.lower()
            print("✅ Fixed column names:", df.columns.tolist())

            # 🔹 Use DELETE instead of TRUNCATE for transactional safety
            print(f"⚠️ Deleting existing records from: {table_name}")
            conn.execute(text(f"DELETE FROM {table_name};"))
            print(f"✅ Table '{table_name}' cleared.\n")

        # 📥 Insert Data Efficiently
        print(f"📥 Inserting data into {table_name} table...")
        df.to_sql(name=table_name, con=engine, if_exists="append", index=False, method="multi", chunksize=1000)

        print(f"✅ Data successfully inserted into '{table_name}' table.\n")

    except Exception as e:
        logging.error(f"❌ Error updating database: {e}", exc_info=True)
        print(f"❌ Error occurred while updating database: {e}")

    finally:
        engine.dispose()  # Ensure connection is closed



In [ ]:
## **Main Execution Flow*
def main():
    session = login_to_website()
    if session is None:
        return
    
    df = fetch_all_concession_tables(session)
    if df is None or df.empty:
        print("❌ No data found! The page structure might have changed.")
        return
    
    print("✅ Data extracted successfully! Cleaning data...\n")
    df = clean_data(df)
    print("✅ Data cleaned successfully!\n")
    
    print("✅ Columns are:\n", df.columns)
    print(df.dtypes)

    
    output_file = r"D:\\GITHUB\\kotak-school-dbms\\output_data\\fee_concession_report_2024_25.csv"
    df.to_csv(output_file, index=False)
    print(f"✅ Data saved to '{output_file}'\n")
    
    update_database(df, TABLE_NAME, POSTGRES_CREDENTIALS)
    print(f"✅ {len(df)} records entered into the database")
    
    print(df.to_string())

## **Run the Main Function*
if __name__ == "__main__":
    main()

✅ Login successful!

✅ Data extracted successfully! Cleaning data...

✅ Data cleaned successfully!

✅ Columns are:
 Index(['id', 'date', 'student_number', 'student_name', 'discount_given'], dtype='object')
id                  int64
date               object
student_number     object
student_name       object
discount_given    float64
dtype: object
✅ Data saved to 'D:\\GITHUB\\kotak-school-dbms\\output_data\\fee_concession_report_2024_25.csv'

🔄 Connecting to database schooldb at localhost...
✅ Fixed column names: ['id', 'date', 'student_number', 'student_name', 'discount_given']
⚠️ Deleting existing records from: fee_concession_2024_25
✅ Table 'fee_concession_2024_25' cleared.

📥 Inserting data into fee_concession_2024_25 table...
✅ Data successfully inserted into 'fee_concession_2024_25' table.

✅ 122 records entered into the database
      id        date student_number                                           student_name  discount_given
0      1  2024-07-27          15660          